# Apache Spark Koalas

### 1. Introduccion a Koalas

Koalas es una API de pandas programada sobre Apache Spark. Tiene todas las ventajas de la implementación de los DataFrames de Spark para trabajar en clúster, pero utilizando la sintaxis de pandas.

#### Beneficios de Koalas
Koalas facilita que los Data Scientists familiarizados con pandas puedan trabajar casi de inmediato en entornos Big Data con Spark, aprendiendo a hacerlo de forma mucho más rápida y sencilla.

Además, permite emplear una única librería base para trabajar con conjuntos de datos de cualquier tamaño, en vez de tener que utilizar pandas para datasets pequeños y PySpark para grandes.

#### Ejemplos de uso de Koalas
Este notebook contiene las funciones principales de Koalas, obtenidas de la documentación oficial de https://koalas.readthedocs.io/

In [2]:
import sys
sys.executable

'C:\\Users\\layla\\anaconda3\\python.exe'

In [3]:
!pip install koalas

In [4]:
import findspark
findspark.init()

import pandas as pd
import pyspark

import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession

### 2. Creación de objetos

Creando una serie Koalas pasando una lista de valores, permitiendo que Koalas cree un índice entero predeterminado:

In [5]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [6]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Creando un Koalas DataFrame pasando un dict de objetos que se pueden convertir a series.

In [7]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [8]:
kdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Creando un DataFrame de pandas pasando una matriz numpy, con un índice de fecha y hora y columnas etiquetadas:

In [9]:
dates = pd.date_range('20130101', periods=6)

In [10]:
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [11]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [12]:
pdf

,A,B,C,D
2013-01-01,0.296152,-0.215817,-0.391997,0.118745
2013-01-02,0.044379,0.349173,-0.188457,0.066455
2013-01-03,0.744525,0.265660,-0.251066,0.677190
2013-01-04,-0.489116,1.277547,0.304424,-2.593397
2013-01-05,0.116915,1.120170,-0.365417,0.465095
2013-01-06,-0.474115,1.429882,-0.867580,-1.035799


Ahora, este DataFrame de pandas se puede convertir en un DataFrame de Koalas

In [13]:
kdf = ks.from_pandas(pdf)

In [14]:
type(kdf)

databricks.koalas.frame.DataFrame

Sin embargo, se ve y se comporta igual que un DataFrame de pandas

In [15]:
kdf

,A,B,C,D
2013-01-01,0.296152,-0.215817,-0.391997,0.118745
2013-01-02,0.044379,0.349173,-0.188457,0.066455
2013-01-03,0.744525,0.265660,-0.251066,0.677190
2013-01-04,-0.489116,1.277547,0.304424,-2.593397
2013-01-05,0.116915,1.120170,-0.365417,0.465095
2013-01-06,-0.474115,1.429882,-0.867580,-1.035799


Además, es posible crear un **Koalas DataFrame desde Spark DataFrame**.

Creando un Spark DataFrame a partir de pandas DataFrame

In [16]:
spark = SparkSession.builder.getOrCreate()

In [17]:
sdf = spark.createDataFrame(pdf)

In [18]:
sdf.show()

+--------------------+--------------------+--------------------+-------------------+
|                   A|                   B|                   C|                  D|
+--------------------+--------------------+--------------------+-------------------+
| 0.29615231058572056|-0.21581669345913104| -0.3919973355730924|0.11874484580111531|
|0.044379013962365974| 0.34917316342486104|-0.18845664139431073|0.06645488760730403|
|  0.7445246714017707|  0.2656600343550899| -0.2510658115353583| 0.6771898706194357|
|-0.48911595381271716|   1.277546934890121|  0.3044237066145135| -2.593396542357387|
| 0.11691548345955091|  1.1201699234737166| -0.3654170537258906| 0.4650945694820092|
|-0.47411468152149955|   1.429882011214831| -0.8675798849779087|-1.0357987764791312|
+--------------------+--------------------+--------------------+-------------------+



Creando Koalas DataFrame desde Spark DataFrame.
`to_koalas ()` se adjunta automáticamente a Spark DataFrame y está disponible como una API cuando se importa Koalas.

In [19]:
kdf = sdf.to_koalas()

In [20]:
kdf

,A,B,C,D
0,0.296152,-0.215817,-0.391997,0.118745
1,0.044379,0.349173,-0.188457,0.066455
2,0.744525,0.265660,-0.251066,0.677190
3,-0.489116,1.277547,0.304424,-2.593397
4,0.116915,1.120170,-0.365417,0.465095
5,-0.474115,1.429882,-0.867580,-1.035799


Tiene [dtypes] específicos. Actualmente se admiten los tipos que son comunes a Spark y pandas.

In [21]:
kdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

### 3. Manipulación de datos


A diferencia de los pandas, los datos en un dataframe de datos de Spark no están _ordenados_, no tienen una noción intrínseca de índice. Cuando se le solicite el encabezado, Spark solo tomará el número solicitado de filas de una partición. **No hay que utilizar el df de Koalas para devolver filas específicas**, use `.loc` o` iloc` en su lugar.

In [22]:
kdf.head()

,A,B,C,D
0,0.296152,-0.215817,-0.391997,0.118745
1,0.044379,0.349173,-0.188457,0.066455
2,0.744525,0.265660,-0.251066,0.677190
3,-0.489116,1.277547,0.304424,-2.593397
4,0.116915,1.120170,-0.365417,0.465095


Muestre el índice, las columnas y los datos numéricos subyacentes.

También puede recuperar el índice; la columna de índice se puede atribuir a un DataFrame, ver más adelante

In [23]:
kdf.index

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [24]:
kdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [25]:
kdf.to_numpy()

array([[ 0.29615231, -0.21581669, -0.39199734,  0.11874485],
       [ 0.04437901,  0.34917316, -0.18845664,  0.06645489],
       [ 0.74452467,  0.26566003, -0.25106581,  0.67718987],
       [-0.48911595,  1.27754693,  0.30442371, -2.59339654],
       [ 0.11691548,  1.12016992, -0.36541705,  0.46509457],
       [-0.47411468,  1.42988201, -0.86757988, -1.03579878]])

**Describe** muestra un resumen estadístico rápido de sus datos

In [26]:
kdf.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,0.039790,0.704436,-0.293349,-0.383619
std,0.471632,0.662294,0.378098,1.233614
min,-0.489116,-0.215817,-0.867580,-2.593397
25%,-0.474115,0.265660,-0.391997,-1.035799
50%,0.044379,0.349173,-0.365417,0.066455
75%,0.296152,1.277547,-0.188457,0.465095
max,0.744525,1.429882,0.304424,0.677190


Transposición de sus datos

In [27]:
kdf.T

,0,1,2,3,4,5
A,0.296152,0.044379,0.744525,-0.489116,0.116915,-0.474115
B,-0.215817,0.349173,0.265660,1.277547,1.120170,1.429882
C,-0.391997,-0.188457,-0.251066,0.304424,-0.365417,-0.867580
D,0.118745,0.066455,0.677190,-2.593397,0.465095,-1.035799


Ordenando por su índice

In [28]:
kdf.sort_index(ascending=False)

,A,B,C,D
5,-0.474115,1.429882,-0.867580,-1.035799
4,0.116915,1.120170,-0.365417,0.465095
3,-0.489116,1.277547,0.304424,-2.593397
2,0.744525,0.265660,-0.251066,0.677190
1,0.044379,0.349173,-0.188457,0.066455
0,0.296152,-0.215817,-0.391997,0.118745


Ordenar por valor

In [29]:
kdf.sort_values(by='B')

,A,B,C,D
0,0.296152,-0.215817,-0.391997,0.118745
2,0.744525,0.265660,-0.251066,0.677190
1,0.044379,0.349173,-0.188457,0.066455
4,0.116915,1.120170,-0.365417,0.465095
3,-0.489116,1.277547,0.304424,-2.593397
5,-0.474115,1.429882,-0.867580,-1.035799


#### Si te interesa saber mas sobre esta libreria, puedes apoyarte en el siguiente link:
* https://github.com/databricks/koalas